<a href="https://colab.research.google.com/github/uditbhaskar21/PySpark/blob/main/startdate_enddate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=6b46482bddcaf1bf6d5aaacb417a59bf92361488d05dd9a4011aed65a7469fe1
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()
print(spark.version)


3.4.1


In [15]:
from pyspark.sql.functions import *


In [16]:
data = [
    ('2020-06-01','Win'),
    ('2020-06-02','Win'),
    ('2020-06-03','Win'),
    ('2020-06-04','Lost'),
    ('2020-06-05','Lost'),
    ('2020-06-06','Lost'),
    ('2020-06-07','Win'),
    ('2020-06-08','Lost'),
]

In [17]:
df = spark.createDataFrame(data,['event_date','status'])

In [18]:
df = df.withColumn('event_date', to_date(col('event_date')))

In [20]:
df.show()

+----------+------+
|event_date|status|
+----------+------+
|2020-06-01|   Win|
|2020-06-02|   Win|
|2020-06-03|   Win|
|2020-06-04|  Lost|
|2020-06-05|  Lost|
|2020-06-06|  Lost|
|2020-06-07|   Win|
|2020-06-08|  Lost|
+----------+------+



In [21]:
from pyspark.sql.window import Window

In [22]:
df1 = df.withColumn("event_change", when(col("status")!= lag("status").over(Window.orderBy("event_date")),1).otherwise(0))

In [23]:
df1.show()

+----------+------+------------+
|event_date|status|event_change|
+----------+------+------------+
|2020-06-01|   Win|           0|
|2020-06-02|   Win|           0|
|2020-06-03|   Win|           0|
|2020-06-04|  Lost|           1|
|2020-06-05|  Lost|           0|
|2020-06-06|  Lost|           0|
|2020-06-07|   Win|           1|
|2020-06-08|  Lost|           1|
+----------+------+------------+



In [24]:
df2 = df1.withColumn("event_group", sum("event_change").over(Window.orderBy("event_date")))

In [25]:
df2.show()

+----------+------+------------+-----------+
|event_date|status|event_change|event_group|
+----------+------+------------+-----------+
|2020-06-01|   Win|           0|          0|
|2020-06-02|   Win|           0|          0|
|2020-06-03|   Win|           0|          0|
|2020-06-04|  Lost|           1|          1|
|2020-06-05|  Lost|           0|          1|
|2020-06-06|  Lost|           0|          1|
|2020-06-07|   Win|           1|          2|
|2020-06-08|  Lost|           1|          3|
+----------+------+------------+-----------+



In [27]:
df3 = df2.groupBy("event_group","status").agg(first("event_date").alias("start_date"), last("event_date").alias("end_date")).drop("event_group")

In [28]:
df3.show()

+------+----------+----------+
|status|start_date|  end_date|
+------+----------+----------+
|   Win|2020-06-01|2020-06-03|
|  Lost|2020-06-04|2020-06-06|
|   Win|2020-06-07|2020-06-07|
|  Lost|2020-06-08|2020-06-08|
+------+----------+----------+



Using Spark SQL

In [30]:
from py4j.java_gateway import select
with cte1 as (
    select
    event_date,
    status,
    CASE WHEN event_status != LAG(event_status) OVER( order by event_date) THEN 1 ELSE 0 END AS event_change
    from table)

, cte 2 as (
    select
    event_date, status, SUM(event_change) over (order by event_date) as event_group
    from CTE1
)

Group by event_group, event_status
order by event_group


SyntaxError: ignored